## Install and import dependencies

In [ ]:
!pip install ibm_watson

In [1]:
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## Setup STT Service

In [2]:
apikey = 'eNENNFauHn-tUxJTL4SEVpsZl87d02XFRMNpEqM7CkeZ'
url = 'https://api.eu-de.speech-to-text.watson.cloud.ibm.com/instances/7db0794e-a4ff-4680-8b34-888eac3adeb1'

In [3]:
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## Compressing and spliting up the Audio

In [16]:
import subprocess
import os
command = 'ffmpeg -i audio.wav -vn -ar 44100 -ac 2 -b:a 192k audio.mp3'
subprocess.call(command, shell = True)
command = 'ffmpeg -i audio.mp3 -f segment -segment_time 360 -c copy %03d.mp3'
subprocess.call(command, shell = True)

0

In [17]:
files = []
for filename in os.listdir('.'):
    if filename.endswith(".mp3") and filename !='audio.mp3':
        files.append(filename)
files.sort()

In [18]:
files

['000.mp3']

## Perform Conversion

In [19]:
results = []
for filename in files:
    with open(filename, 'rb') as f:
        res = stt.recognize(audio=f, content_type='audio/mp3', model='en-AU_NarrowbandModel', continuous=True, \
                           inactivity_timeout=360).get_result()
        results.append(res)

In [20]:
results[0]

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'thank you for choosing the Olympic Tatian management system ',
     'confidence': 0.83}]},
  {'final': True,
   'alternatives': [{'transcript': 'the Olympus dictation management system gives you the power to manage your dictation transcriptions and documents seamlessly and to improve the productivity of your daily work for example you can automatically sent the dictation files or transcribed documents to your assistant or the also via email or F. T. P. ',
     'confidence': 0.94}]},
  {'final': True,
   'alternatives': [{'transcript': "if you're using the speech recognition software the speech recognition engine works in the background to support your document creation ",
     'confidence': 0.98}]},
  {'final': True,
   'alternatives': [{'transcript': 'we hope you enjoy the simple flexible reliable and secure solutions from Olympus thank you for choosing the Olympic Tatian management system ',
     'co

In [21]:
text = []
for file in results:
    for result in file['results']:
        text.append(result['alternatives'][0]['transcript'].rstrip() + '.\n')

In [22]:
text

['thank you for choosing the Olympic Tatian management system.\n',
 'the Olympus dictation management system gives you the power to manage your dictation transcriptions and documents seamlessly and to improve the productivity of your daily work for example you can automatically sent the dictation files or transcribed documents to your assistant or the also via email or F. T. P..\n',
 "if you're using the speech recognition software the speech recognition engine works in the background to support your document creation.\n",
 'we hope you enjoy the simple flexible reliable and secure solutions from Olympus thank you for choosing the Olympic Tatian management system.\n',
 'the Olympus dictation management system gives you the power to manage your dictation transcriptions and documents seamlessly and to improve the productivity of your daily work for example you can automatically sent the dictation files or transcribed documents to your assistant or the also via email or F. T. P..\n',
 "if

In [23]:
with open('output.txt', 'w') as out:
    out.writelines(text)